In [2]:
!git clone -b aot/head_dim_rope --single-branch https://github.com/suiyoubi/transformers.git && cd transformers
!pip install -e ./transformers/.
!pip install scikit-learn

Cloning into 'transformers'...
Updating files:  65% (2862/4381)
Updating files:  66% (2892/4381)
Updating files:  67% (2936/4381)
Updating files:  68% (2980/4381)
Updating files:  69% (3023/4381)
Updating files:  70% (3067/4381)
Updating files:  71% (3111/4381)
Updating files:  72% (3155/4381)
Updating files:  73% (3199/4381)
Updating files:  74% (3242/4381)
Updating files:  75% (3286/4381)
Updating files:  76% (3330/4381)
Updating files:  77% (3374/4381)
Updating files:  78% (3418/4381)
Updating files:  79% (3461/4381)
Updating files:  80% (3505/4381)
Updating files:  81% (3549/4381)
Updating files:  82% (3593/4381)
Updating files:  83% (3637/4381)
Updating files:  84% (3681/4381)
Updating files:  85% (3724/4381)
Updating files:  86% (3768/4381)
Updating files:  87% (3812/4381)
Updating files:  88% (3856/4381)
Updating files:  89% (3900/4381)
Updating files:  90% (3943/4381)
Updating files:  91% (3987/4381)
Updating files:  92% (4031/4381)
Updating files:  93% (4075/4381)
Updating fil

Obtaining file:///C:/Users/sequi/Documents/Projets/PrimeIntellect/transformers
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for transformers (pyproject.toml): started
  Building editable for transformers (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.45.0.dev0-0.editable-py3-none-any.whl size=17184 sha256=b9366a989d56aa3783ac04c0f3ac4b664ad21447672f7f952cee4c15925f4832
  Stored in directory: C:\Users\sequi\AppData\Local\Temp\pip-ephem-wheel-cache-vwlpir_2\wheels\aa\09

In [ ]:
# Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

In [ ]:
model_path = "nvidia/Minitron-8B-Base"
device='cuda'
dtype=torch.bfloat16
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto")

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

train_dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
train_dataset = train_dataset.map(formatting_prompts_func, batched = True)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
model.save_pretrained("minitron_model") # Local saving
tokenizer.save_pretrained("minitron_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving